In [3]:
# Credit Card Fraud Detection
# Dataset Link: https://www.kaggle.com/dalpozz/creditcardfraud
# Dataset Size: 68 MB

import os, sys
import pandas as pd
import numpy as np

In [6]:
csv_data = "creditcard.csv"
dataframe = pd.read_csv(csv_data)
print("# of samples: {}\n# of features: {}".format(dataframe.shape[0],dataframe.shape[1]))

# of samples: 284807
# of features: 31
